In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path

# Function to process a single file and return a GeoDataFrame
def process_file_to_geodataframe(data):
    # Group by 2-second intervals and calculate means
    data['time_bin'] = (data['seconds_elapsed'] // 2) * 2
    grouped = data.groupby('time_bin').agg({
        'ax': 'mean',
        'ay': 'mean',
        'az': 'mean',
        'gx': 'mean',
        'gy': 'mean',
        'gz': 'mean',
        'time': 'first',
        'longitude': 'first',
        'latitude': 'first',
        'class': 'first'
    }).reset_index()

    # Create GeoDataFrame
    return gpd.GeoDataFrame(
        grouped,
        geometry=[Point(xy) for xy in zip(grouped.longitude, grouped.latitude)],
        crs="EPSG:4326"
    )

# Function to process all CSV files in a specified folder and save to a single GeoJSON
def process_folder_and_save_single_geojson(folder_path, output_path):
    csv_files = list(Path(folder_path).glob('*.csv'))  # List all CSV files in the folder
    all_gdfs = []  # List to store all GeoDataFrames

    for file_path in csv_files:
        data = pd.read_csv(file_path)
        print(f'Trying {file_path}...')
        try:
            gdf = process_file_to_geodataframe(data)
        except Exception as e:
            print(f'Error:', e)
            continue
        all_gdfs.append(gdf)

    # Concatenate all GeoDataFrames into one
    if all_gdfs:
        final_gdf = pd.concat(all_gdfs)
        final_gdf.to_file(output_path, driver='GeoJSON')

# Example usage:
process_folder_and_save_single_geojson(r'../preprocessed_data', r'../geoproccessed_data/result.geojson')


Trying ..\preprocessed_data\asphalt-2024-06-15_16-07-55.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-13-20.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-16-43.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-17-44.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-18-42.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-19-31.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-30-49.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-31-38.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-32-45.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-33-26.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-36-27.csv...
Trying ..\preprocessed_data\asphalt-2024-06-15_16-37-10.csv...
Error: "Column(s) ['latitude', 'longitude'] do not exist"
Trying ..\preprocessed_data\asphalt-2024-06-15_16-58-46.csv...
Trying ..\preprocessed_data\asphalt_bad-2024-06-15_17-00-41.csv...
Trying ..\preprocessed_data\asphalt_bad-2024-06-15_17-02